In [ ]:
TO DO

general implementation
    create a graph or diagram to tell whats hapenning
    mark all the tasks/steps complete, incomplete, in progress, problems, to do, research etc.
    try on a toy problem

find new datasets
    for different datasets different preprocessing techniques should be applied
    RCV1-V2
decide on splitting ratio 20 60 20 

try different similarity measures 
    reference paper
    cosine
    euclidean

In [ ]:
implementation steps

1. reading data and preprocessing
2. vectorization
    2.1 embeddings
    2.2 dimensionality reduction? (is similarity more accurate when dim. red. done) research
3. initial classifier to show results
4. calculate imbalance ratio and find the ratio of newly labeled data
5. oversample dataset using unlabeled set
    5.1 find the proper similarity function (eclidean, cosine etc.)
        Measurement of Text Similarity: A Survey: a very detailed survey of similarity functions that are used for text data
        https://www.kdnuggets.com/2019/01/comparison-text-distance-metrics.html
        cosine similarity
        minkowski family (euclidean, manhattan)
        hamming distance
        Jaccard index
        Sorensen-dice index
        KL divergence
        Jensen–Shannon divergence with LDA
        Wasserstein distance
        SMTP 
        word mover’s distance
    5.2 define a threshold or mechanism to add data for multilabeled set
6. train a final classifier to compare results

In [37]:
import pandas as pd
import numpy as np
import preprocess

In [38]:
def read_data(path):
    df = pd.read_csv(path)
    df['text'] = df['text'].apply(preprocess.preprocess_text)
    return df

In [39]:
def vectorize_data(text, model_name='stsb-roberta-large'):
    
    from sentence_transformers import util
    from sentence_transformers import SentenceTransformer
    import torch
    
    model = SentenceTransformer(model_name)
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    
    vectors = model.encode(text, convert_to_tensor=False, device=device)
    
    return vectors

In [40]:
def classifier(X_train, y_train, X_test, y_test):
    
    def calculating_class_weights(y_true):
        from sklearn.utils.class_weight import compute_class_weight
        number_dim = np.shape(y_true)[1]
        print(number_dim)
        weights = []
        for i in range(number_dim):
            print(y_true[:, i])
            at = compute_class_weight('balanced', [0.,1.], y_true[:, i])
            weights.append(dict(zip([0,1], at)))
            # weights[i] = compute_class_weight('balanced', [0.,1.], y_true[:, i])))
        return weights

    class_weights = calculating_class_weights(y_train.values)
    
    # Linear SVM
    linearSvm = OneVsRestClassifier(LinearSVC(class_weight='balanced'), n_jobs=-1)
    linearSvm.fit(X_train, y_train.values)
    linearSvm_preds = linearSvm.predict(X_test)
    print_losses(y_test, linearSvm_preds, 'Linear SVM Classifier')
    
    print("\033[1m" + 'LinearSVM results: ' + "\033[0m")
    print('-'*30)
    hamLoss = hamming_loss(y_test.values, linearSvm_preds)
    print('hamLoss: {:.2f}'.format(hamLoss))
    acc_score = accuracy_score(y_test.values, linearSvm_preds)
    print('Exact Match Ratio: {:.2f}'.format(acc_score))
    print('-'*30)
    print("\033[1m" + 'Classification Report' + "\033[0m")
    print(classification_report(y_test.values, linearSvm_preds, target_names=list(y_test.columns)))

In [41]:
y_labeled.values[:, 1]

NameError: name 'y_labeled' is not defined

In [ ]:
classifier(X_labeled, y_labeled, X_test, y_test)

In [43]:
def find_balancing_ratio(y):
    balancing_ratio = 1.0
    return balancing_ratio

In [44]:
def oversample_dataset(labeled_data, unlabeled_data):
    return labeled_data, unlabeled_data

# try

In [45]:
majority_path = r'C:\Users\IsmailKaraman\workspace\data\privacy_policy_data\OPP-115_v2\majority.csv'
df = pd.read_csv(majority_path)
df.head()

,text,Data Retention,Data Security,Do Not Track,First Party Collection/Use,International and Specific Audiences,Introductory/Generic,Policy Change,Practice not covered,Privacy contact information,Third Party Sharing/Collection,"User Access, Edit and Deletion",User Choice/Control
0,"""""Contact Us"""" Link If you contact us through...",0,0,0,1,0,0,0,0,0,0,0,0
1,(b) Information automatically collected There...,0,0,0,1,0,0,0,0,0,0,0,0
2,(ii) You have entered a contest or sweepstake...,0,0,0,0,0,0,0,0,0,1,0,0
3,*Web Beacons: Military Web pages and the Web ...,0,0,0,1,0,0,0,0,0,1,0,0
4,1. INFORMATION WE COLLECT We collect the foll...,0,0,0,0,0,1,0,0,0,0,0,0


In [46]:
col_names = ['Data Retention', 'Data Security', 'Do Not Track', 'First Party Collection/Use', 
             'International and Specific Audiences',       'Introductory/Generic', 'Policy Change', 
             'Practice not covered', 'Privacy contact information', 'Third Party Sharing/Collection',
             'User Access, Edit and Deletion', 'User Choice/Control']
sub_col_names = ['Data Security', 'User Access, Edit and Deletion', 'Policy Change']

In [47]:
toy_df = df[(df[col_names].sum(axis=1)==df[sub_col_names].sum(axis=1))].sample(100)

In [48]:
toy_df.sum()

text                                    Material that is, really important changes to ...
Data Retention                                                                          0
Data Security                                                                          46
Do Not Track                                                                            0
First Party Collection/Use                                                              0
International and Specific Audiences                                                    0
Introductory/Generic                                                                    0
Policy Change                                                                          29
Practice not covered                                                                    0
Privacy contact information                                                             0
Third Party Sharing/Collection                                                          0
User Acces

In [49]:
X = toy_df['text']
y = toy_df[sub_col_names]

In [ ]:
X_num = X.apply(vectorize_data)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
X_labeled, X_unlabeled, y_labeled, y_unlabeled = train_test_split(X_train, y_train, test_size=0.75, stratify=y, random_state=2)

In [ ]:
X_labeled.shape, X_unlabeled.shape, X_test.size

In [ ]:
y_labeled.sum()

In [70]:
classifier(X_labeled, y_labeled, X_test, y_test)

3


TypeError: compute_class_weight() takes 1 positional argument but 3 were given

In [65]:
y_labeled.values

array([[0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]], dtype=int64)

In [16]:
from nltk.corpus import words
all_words = words.words()

In [18]:
words_5len = [word.lower() for word in all_words if len(word)==5]

In [35]:
for word in words_5len:
    if word.startswith('s') and word.endswith('e') and word[1]!='r' and 'r' in word and 'c' not in word and 'a' not in word\
    and 'i' not in word and 'p' in word:
        print(word)

spore
spree
sprue


In [34]:
for word in words_5len:
    if 'o' in word and 'u' in word and 'a' not in word and 'i' not in word and 'd' not in word:
        print(word)

blout
bogue
bogum
bogus
bolus
bonus
bosun
bouge
bough
boule
bourg
bourn
bouse
bousy
bouto
bruno
bucco
bucko
bumbo
bungo
bunko
burro
buteo
buxom
byous
choup
chous
clour
clout
cobus
cogue
comus
conus
copus
coque
cornu
couch
cough
count
coupe
courb
cours
court
couth
coypu
croup
crout
cruor
cumol
cusso
epulo
eusol
flour
flout
fluor
focus
fogou
fogus
forum
fount
foute
fouth
fungo
furor
ghoul
glout
gouge
gouty
grouf
group
grout
gubbo
gumbo
gusto
hocus
hokum
hough
house
housy
hucho
humbo
humor
huron
huron
husho
jocum
jokul
jorum
jough
joule
jours
joust
jumbo
junco
junto
jupon
juror
justo
knout
kobus
kokum
kombu
kongu
leuco
locum
locus
lorum
lotus
louch
louey
lough
loulu
loupe
louse
lousy
louty
luteo
mogul
mohur
mokum
momus
montu
mopus
morus
mosgu
mosul
mosur
moule
mouls
mouly
mount
mourn
mouse
mousy
mouth
mucor
mucro
mungo
muong
notum
notus
nullo
occur
ocque
ocuby
oecus
oghuz
okrug
onmun
orgue
ought
ounce
ouphe
outby
outen
outer
outgo
outly
outre
ouzel
ovule
plouk
plout
pluto
porus
pouce
pouc